In [ ]:
#lastest code'
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results3"
os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        # lower to detect small axons
MIN_CONTOUR_AREA = 8          # reduced from 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           # reduced from 0.3

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):

    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Small smoothing for better boundary separation
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(smooth)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # NEW: Separate touching axons using WATERSHED
    # ---------------------------
    # thresh: foreground (axon structures) = 255, background = 0
    kernel = np.ones((3, 3), np.uint8)

    # Remove tiny noise, keep main blobs
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground via distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    # Unknown region (possible boundaries between axons)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling for watershed
    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1  # background becomes 1 instead of 0
    markers[unknown == 255] = 0

    # Watershed needs 3-channel image
    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    # Use watershed boundaries to carve gaps between touching axons
    cleaned = thresh.copy()
    cleaned[markers == -1] = 0  # make watershed boundary black (gap)
    # ---------------------------

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):

        # Only consider outermost contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # inner contours
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    print(f"Processed {len(axon_data)} axons.")
    return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        axon_data, output_image = process_image(img_path)

        if axon_data is None:
            continue

        df = save_to_csv(axon_data, img_output_folder)
        plot_and_visualize(df, img_output_folder)

        out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        cv2.imwrite(out_img_path, output_image)

        print(f"✔ Saved results for {img_name} in {img_output_folder}\n")

# --------------------------
# RUN
# --------------------------
process_folder(input_folder, output_root)

print("\n🎉 DONE! All images processed.")


In [ ]:
#code with denoise and deblur
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# CONFIG: Give folder containing input images
# --------------------------
input_folder = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02"
output_root = r"D:\projects\neuro\all fascicles\set2_Image_01_20x_bf_02_results4"
os.makedirs(output_root, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables (UPDATED)
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245

MIN_CIRCULARITY = 0.05        
MIN_CONTOUR_AREA = 8          
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.15           

# --------------------------
# NEW: DENOISE + DEBLUR
# --------------------------
def denoise_image(img):
    """Fast Non-Local Means denoising (color)."""
    return cv2.fastNlMeansDenoisingColored(img, None, 7, 7, 7, 21)

def wiener_deblur(gray, ksize=3, noise_power=0.005):
    """Safe Wiener deblurring filter."""
    gray_f = np.float32(gray) / 255.0

    psf = np.ones((ksize, ksize), np.float32) / (ksize * ksize)

    G = np.fft.fft2(gray_f)
    H = np.fft.fft2(psf, s=gray_f.shape)
    H_conj = np.conj(H)
    denom = (H * H_conj) + noise_power

    F_hat = (H_conj / denom) * G
    out = np.abs(np.fft.ifft2(F_hat))
    out = np.clip(out, 0, 1)
    return np.uint8(out * 255)

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):

    # -------------------------------------
    # NEW: DENOISE + DEBLUR BEFORE ANYTHING
    # -------------------------------------
    patch = denoise_image(patch)       # safe denoising
    gray0 = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
    gray0 = wiener_deblur(gray0)       # safe deblurring
    patch = cv2.cvtColor(gray0, cv2.COLOR_GRAY2BGR)

    # Sharpen (from your original code)
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Small smoothing
    smooth = cv2.bilateralFilter(gray, 5, 20, 20)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(smooth)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )

    thresh = cv2.bitwise_or(otsu, adaptive)

    # ---------------------------
    # WATERSHED
    # ---------------------------
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    if dist_transform.max() > 0:
        _, sure_fg = cv2.threshold(dist_transform,
                                   0.4 * dist_transform.max(), 255, 0)
    else:
        sure_fg = np.zeros_like(opening)
    sure_fg = np.uint8(sure_fg)

    unknown = cv2.subtract(sure_bg, sure_fg)

    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    wshed_input = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(wshed_input, markers)

    cleaned = thresh.copy()
    cleaned[markers == -1] = 0

    contours, hierarchy = cv2.findContours(
        cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    print(f"Processed {len(axon_data)} axons.")
    return axon_data, output_image

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        axon_data, output_image = process_image(img_path)

        if axon_data is None:
            continue

        df = save_to_csv(axon_data, img_output_folder)
        plot_and_visualize(df, img_output_folder)

        out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        cv2.imwrite(out_img_path, output_image)

        print(f"✔ Saved results for {img_name} in {img_output_folder}\n")

# --------------------------
# RUN
# --------------------------
process_folder(input_folder, output_root)
print("\n🎉 DONE! All images processed.")


In [ ]:
#old best code
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# Input and output paths
# --------------------------
image_path = r"C:\Users\DELL\Downloads\fullImages\fullImages\set1_Image_01_40x_bf_02.png"
output_folder = r"C:\Users\DELL\Downloads\40x_bf_02_qwerty5_4_statistics_regrowth_all"
os.makedirs(output_folder, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245
MIN_CIRCULARITY = 0.10
MIN_CONTOUR_AREA = 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.3  # adjust based on dataset

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)
    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # Use both Otsu + adaptive threshold (union of both)
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )
    thresh = cv2.bitwise_or(otsu, adaptive)

    # Clean
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):
        # only outer contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # Collect all inner children
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        # Classification
        axon_type = "mature" if len(inner_children) == 1 else "regrowth_cluster"

        # Largest inner for measurement
        largest_inner = max(inner_children, key=cv2.contourArea)

        # --- Outer ---
        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset
        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor**2)

        # Draw outer boundary with type-specific color
        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        # --- Inner ---
        # --- Inner ---
        inner_radii = []
        inner_areas = []
        
        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)  # Draw all in green
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))
        
        # Use the largest inner contour for metrics
        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        # Metrics
        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = (inner_area_um2 / outer_area_um2) if outer_area_um2 > 0 else 0
        hole_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        # Create masks for mean intensity
        mask_outer = np.zeros(gray.shape, dtype=np.uint8)
        mask_inner = np.zeros(gray.shape, dtype=np.uint8)
        cv2.drawContours(mask_outer, [contour], -1, 255, -1)
        cv2.drawContours(mask_inner, [largest_inner], -1, 255, -1)

        outer_mean = cv2.mean(gray, mask=mask_outer)[0]
        inner_mean = cv2.mean(gray, mask=mask_inner)[0]
        ring_contrast = (outer_mean - inner_mean) / (outer_mean + 1e-6)

        # LAB a-channel mean inside axon center
        lab = cv2.cvtColor(patch, cv2.COLOR_BGR2LAB)
        _, A, _ = cv2.split(lab)
        a_mean_center = cv2.mean(A, mask=mask_inner)[0]

        # Eccentricity
        eccentricity = 0
        if len(contour) >= 5:
            ellipse = cv2.fitEllipse(contour)
            (MA, ma) = ellipse[1]
            if MA > 0:
                eccentricity = np.sqrt(1 - (ma / MA) ** 2)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": hole_ratio,
            "ring_contrast": ring_contrast,
            "a_mean_center": a_mean_center,
            "eccentricity": eccentricity,
            "g_ratio": g_ratio
        })

        # Numbering
        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        print(f"Axon {object_counter} ({axon_type}):")
        print(f" Center = ({x_outer:.2f}, {y_outer:.2f})")
        print(f" Outer Radius = {outer_radius*scale_factor:.4f} µm")
        print(f" Inner Radius = {inner_radius*scale_factor:.4f} µm")
        print(f" Thickness = {thickness:.4f} µm")
        print(f" Diameter = {(2*outer_radius)*scale_factor:.4f} µm")
        print(f" Outer Area = {outer_area_um2:.4f} µm²")
        print(f" Inner Area = {inner_area_um2:.4f} µm²")
        print(f" G-Ratio = {g_ratio:.4f}")
        print(f" Inner Contours = {len(inner_children)}\n")

        object_counter += 1

    return object_counter

# --------------------------
# Whole image tiling
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Could not read image")

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]
    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f"numbered_{filename}")
    cv2.imwrite(output_path, output_image)

    print(f"\nProcessed: {filename}")
    print(f"Axons detected: {len(axon_data)}\n")
    return axon_data, output_image

# --------------------------
# Save data to CSV
# --------------------------
def save_to_csv(axon_data, output_folder, filename="axon_measurements.csv"):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, filename)
    df.to_csv(csv_path, index=False)
    print(f"✅ Axon data saved to: {csv_path}")
    return df

# --------------------------
# Global Histograms
# --------------------------
def plot_global_histograms(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        data = df[col].dropna().values
        if len(data) == 0 or not np.isfinite(data).any():
            continue

        plt.figure(figsize=(6, 4))
        plt.hist(data, bins=30, color="skyblue", edgecolor="black")
        plt.title(f"Global Histogram of {col}")
        plt.xlabel(col)
        plt.ylabel("Frequency")
        plt.grid(True, linestyle="--", alpha=0.6)

        hist_path = os.path.join(output_folder, f"hist_global_{col}.png")
        plt.savefig(hist_path, dpi=150, bbox_inches="tight")
        plt.close()

# --------------------------
# Box, Violin, Scatter Plots
# --------------------------
def visualize_parameters(df, output_folder):
    param_units = {
        "outer_radius": "Outer Radius (µm)",
        "inner_radius": "Inner Radius (µm)",
        "thickness": "Thickness (µm)",
        "diameter": "Diameter (µm)",
        "outer_area": "Outer Area (µm²)",
        "inner_area": "Inner Area (µm²)",
        "area_ratio": "Area Ratio",
        "hole_ratio": "Hole Ratio",
        "ring_contrast": "Ring Contrast",
        "a_mean_center": "A-Mean Center (intensity)",
        "eccentricity": "Eccentricity",
        "g_ratio": "g-ratio",
        "num_inner_contours": "Number of Inner Contours"
    }

    df = df.copy()
    df = df.apply(pd.to_numeric, errors="ignore")

    # 1️⃣ Print all data
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    print("\n📋 Axon Data (All rows):")
    print(df)

    # 2️⃣ Individual plots
    for param, label in param_units.items():
        if param not in df:
            continue
        data = df[param].dropna()
        if len(data) == 0:
            continue

        # Boxplot
        plt.figure(figsize=(6, 4))
        sns.boxplot(x=data, color="skyblue")
        plt.title(f"Boxplot of {label}")
        plt.xlabel(label)
        plt.savefig(os.path.join(output_folder, f"box_{param}.png"), dpi=150, bbox_inches="tight")
        plt.close()

        # Violin plot
        plt.figure(figsize=(6, 4))
        sns.violinplot(x=data, inner="box", color="lightgreen")
        plt.title(f"Violin Plot of {label}")
        plt.xlabel(label)
        plt.savefig(os.path.join(output_folder, f"violin_{param}.png"), dpi=150, bbox_inches="tight")
        plt.close()

        # Histogram
        plt.figure(figsize=(6, 4))
        sns.histplot(data, kde=True, bins=30, color="blue")
        plt.title(f"Histogram of {label}")
        plt.xlabel(label)
        plt.ylabel("Count")
        plt.savefig(os.path.join(output_folder, f"hist_{param}.png"), dpi=150, bbox_inches="tight")
        plt.close()

    # Comparative boxplot
    df_melted = df[list(param_units.keys())].melt(var_name="Parameter", value_name="Value")
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Parameter", y="Value", data=df_melted)
    plt.xticks(rotation=45, ha="right")
    plt.title("Comparative Boxplots of Axon Parameters")
    plt.ylabel("Value (with units depending on parameter)")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, "comparative_boxplots.png"), dpi=150)
    plt.close()

    # Scatter plot matrix
    clean_df = df[list(param_units.keys())].dropna()
    if not clean_df.empty:
        pairplot = sns.pairplot(clean_df, corner=True, plot_kws={"alpha": 0.5, "s": 20})
        pairplot.fig.suptitle("Scatter Plot Matrix of Axon Parameters", y=1.02)
        pairplot.savefig(os.path.join(output_folder, "scatter_matrix.png"), dpi=150, bbox_inches="tight")
        plt.close("all")

    print(f"\n✅ Plots saved in '{output_folder}'")

# --------------------------
# Summary counts
# --------------------------
def save_summary(df, output_folder):
    mature_count = (df["axon_type"] == "mature").sum()
    regrowth_clusters = (df["axon_type"] == "regrowth_cluster").sum()
    total_regrowth_axons = df.loc[df["axon_type"] == "regrowth_cluster", "num_inner_contours"].sum()

    summary_text = (
        f"Number of mature axons: {mature_count}\n"
        f"Number of regrowth clusters: {regrowth_clusters}\n"
        f"Number of total regrowth axons: {total_regrowth_axons}\n"
    )

    print("\n📊 Summary:\n" + summary_text)

    with open(os.path.join(output_folder, "summary.txt"), "w") as f:
        f.write(summary_text)

# --------------------------
# Run Pipeline
# --------------------------
axon_data, output_image = process_image(image_path, patch_size=1024)
df = save_to_csv(axon_data, output_folder)
plot_global_histograms(df, output_folder)
visualize_parameters(df, output_folder)
save_summary(df, output_folder)

print(f"\n✅ All CSV + plots + summary saved in '{output_folder}'")